#import libraries

In [ ]:
import pandas as pd
import numpy as np 
import re
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import svm
from sklearn.metrics import accuracy_score

#import datasets

In [ ]:
train_df=pd.read_csv('Train-Table 1.csv')
train_df.head()

,login,location,skills,evaluation_status,internship_id
0,c2,Chennai,"Digital Circuits,Python,Java,C programming,MIC...",SELECTED,p137
1,c4,Chennai,"C programming,Python,C++",SELECTED,p151
2,c5,Chennai,"Cloud Computing AWS,networking,Digital Circu...",SELECTED,p186
3,c6,Chennai,"Animation and Game Development,Application des...",SELECTED,p186
4,c8,Hyderabad,"C,Java,mysql,Python",REJECTED,p138


In [ ]:
test_df=pd.read_csv('Test-Table 1.csv')
test_df.head()

,candidate_id,location,skills
0,c1,Kolkata,"C,C++,Java,Android Studio,msoffice,Python"
1,c3,Chennai,"networking,C programming,Python"
2,c7,Chennai,"Unity 3D,AR,C sharp,C,JAVA PROGRAMMING,Servlet..."
3,c15,Mumbai,"Javascript,C++,Java,Python,css,HTML,mongoDB,Re..."
4,c16,Kolkata,"C,Core Java,Python,Data Structures Algorithm..."


In [ ]:
train_df=train_df.rename(columns={'login':'candidate_id'})
train_df

,candidate_id,location,skills,evaluation_status,internship_id
0,c2,Chennai,"Digital Circuits,Python,Java,C programming,MIC...",SELECTED,p137
1,c4,Chennai,"C programming,Python,C++",SELECTED,p151
2,c5,Chennai,"Cloud Computing AWS,networking,Digital Circu...",SELECTED,p186
3,c6,Chennai,"Animation and Game Development,Application des...",SELECTED,p186
4,c8,Hyderabad,"C,Java,mysql,Python",REJECTED,p138
...,...,...,...,...,...
943,c814,Hyderabad,"C,Python,Machine learning,Java,HTML,css,sql,js...",REJECTED,p638
944,c814,Hyderabad,"C,Python,Machine learning,Java,HTML,css,sql,js...",REJECTED,p667
945,c814,Hyderabad,"C,Python,Machine learning,Java,HTML,css,sql,js...",SELECTED,p713
946,c815,Bangalore,"Algorithms and Data Structures,Android App Dev...",SELECTED,p700


### Droping 'evaluation_status' column from train dataset

In [ ]:
train_df=train_df.drop('evaluation_status',axis=1)
train_df

,candidate_id,location,skills,internship_id
0,c2,Chennai,"Digital Circuits,Python,Java,C programming,MIC...",p137
1,c4,Chennai,"C programming,Python,C++",p151
2,c5,Chennai,"Cloud Computing AWS,networking,Digital Circu...",p186
3,c6,Chennai,"Animation and Game Development,Application des...",p186
4,c8,Hyderabad,"C,Java,mysql,Python",p138
...,...,...,...,...
943,c814,Hyderabad,"C,Python,Machine learning,Java,HTML,css,sql,js...",p638
944,c814,Hyderabad,"C,Python,Machine learning,Java,HTML,css,sql,js...",p667
945,c814,Hyderabad,"C,Python,Machine learning,Java,HTML,css,sql,js...",p713
946,c815,Bangalore,"Algorithms and Data Structures,Android App Dev...",p700


In [ ]:
train_df.location.unique()

array(['Chennai', 'Hyderabad', 'Kolkata', 'Mumbai', 'Delhi', 'Bangalore',
       'Ahmedabad', 'Pune', 'Trivandrum', 'Lucknow'], dtype=object)

In [ ]:
len(train_df.internship_id.unique())

400

In [ ]:
len(train_df.candidate_id.unique())

577

In [ ]:
train_df.isna().sum()

candidate_id      0
location          0
skills           16
internship_id     0
dtype: int64

In [ ]:
train_df.dropna(subset=['skills'],inplace=True,)
train_df.isna().sum()
train_df = train_df.reset_index(drop=True)

In [ ]:
test_df.isna().sum()


candidate_id    0
location        0
skills          2
dtype: int64

In [ ]:
test_df.dropna(subset=['skills'],inplace=True,)
test_df.isna().sum()
test_df = test_df.reset_index(drop=True)
test_df

,candidate_id,location,skills
0,c1,Kolkata,"C,C++,Java,Android Studio,msoffice,Python"
1,c3,Chennai,"networking,C programming,Python"
2,c7,Chennai,"Unity 3D,AR,C sharp,C,JAVA PROGRAMMING,Servlet..."
3,c15,Mumbai,"Javascript,C++,Java,Python,css,HTML,mongoDB,Re..."
4,c16,Kolkata,"C,Core Java,Python,Data Structures Algorithm..."
...,...,...,...
143,c803,Lucknow,"C++,Data Analytics using Python,HTML,css"
144,c808,Ahmedabad,"algorithm,Data Structure,backend development,W..."
145,c811,Trivandrum,"Python,Java,C,C++"
146,c813,Mumbai,"Python,Java,Deep Learning,C"


In [ ]:
test_df.isna().sum()

candidate_id    0
location        0
skills          0
dtype: int64

In [ ]:
train_df_lowercase=train_df.copy()
test_df_lowercase=test_df.copy()


In [ ]:
train_df_lowercase['skills']=train_df_lowercase['skills'].apply(lambda x: " ".join(x.lower() for x in x.split()))

In [ ]:
test_df_lowercase['skills']=test_df_lowercase['skills'].apply(lambda x: " ".join(x.lower() for x in x.split()))

In [ ]:
train_df_lowercase['location']=train_df_lowercase['location'].apply(lambda x: x.lower() )


In [ ]:
test_df_lowercase['location']=test_df_lowercase['location'].apply(lambda x: x.lower() )


In [ ]:
test_df_lowercase.to_csv("test_df_lowercase.csv")

In [ ]:
test_df_lowercase=pd.read_csv("test_google_sheet.csv")
test_df_lowercase

,candidate_id,location,skills
0,c93,hyderabad,"python,c,java,android studio"
1,c84,pune,"c,java,android app development,python,c++"
2,c816,hyderabad,"c,java,python,sql,machine learning"
3,c813,mumbai,"python,java,deep learning,c"
4,c811,trivandrum,"python,java,c,c++"
...,...,...,...
143,c105,chennai,"c++,c,matlab,java,python,assembly"
144,c102,mumbai,"python,machine learning,android app developmen..."
145,c101,hyderabad,"java,javascript,c c++,dbms sql,reactjs"
146,c100,chennai,"java programming,r programming,c"


In [ ]:
test_df_lowercase.dropna(subset=['skills'],inplace=True,)
test_df_lowercase.isna().sum()

candidate_id    0
location        0
skills          0
dtype: int64

In [ ]:

test_df_lowercase = test_df_lowercase.reset_index(drop=True)
test_df_lowercase

,candidate_id,location,skills
0,c93,hyderabad,"python,c,java,android studio"
1,c84,pune,"c,java,android app development,python,c++"
2,c816,hyderabad,"c,java,python,sql,machine learning"
3,c813,mumbai,"python,java,deep learning,c"
4,c811,trivandrum,"python,java,c,c++"
...,...,...,...
142,c105,chennai,"c++,c,matlab,java,python,assembly"
143,c102,mumbai,"python,machine learning,android app developmen..."
144,c101,hyderabad,"java,javascript,c c++,dbms sql,reactjs"
145,c100,chennai,"java programming,r programming,c"


In [ ]:
test_df_lowercase['features'] = test_df_lowercase[['location', 'skills']].apply(lambda x: ','.join(x), axis=1)
test_df_lowercase

,candidate_id,location,skills,features
0,c93,hyderabad,"python,c,java,android studio","hyderabad,python,c,java,android studio"
1,c84,pune,"c,java,android app development,python,c++","pune,c,java,android app development,python,c++"
2,c816,hyderabad,"c,java,python,sql,machine learning","hyderabad,c,java,python,sql,machine learning"
3,c813,mumbai,"python,java,deep learning,c","mumbai,python,java,deep learning,c"
4,c811,trivandrum,"python,java,c,c++","trivandrum,python,java,c,c++"
...,...,...,...,...
142,c105,chennai,"c++,c,matlab,java,python,assembly","chennai,c++,c,matlab,java,python,assembly"
143,c102,mumbai,"python,machine learning,android app developmen...","mumbai,python,machine learning,android app dev..."
144,c101,hyderabad,"java,javascript,c c++,dbms sql,reactjs","hyderabad,java,javascript,c c++,dbms sql,reactjs"
145,c100,chennai,"java programming,r programming,c","chennai,java programming,r programming,c"


In [ ]:
test_df_lowercase['features_space'] = test_df_lowercase['features'].apply(lambda x: re.split(',| ',x))
test_df_lowercase

,candidate_id,location,skills,features,features_space
0,c93,hyderabad,"python,c,java,android studio","hyderabad,python,c,java,android studio","[hyderabad, python, c, java, android, studio]"
1,c84,pune,"c,java,android app development,python,c++","pune,c,java,android app development,python,c++","[pune, c, java, android, app, development, pyt..."
2,c816,hyderabad,"c,java,python,sql,machine learning","hyderabad,c,java,python,sql,machine learning","[hyderabad, c, java, python, sql, machine, lea..."
3,c813,mumbai,"python,java,deep learning,c","mumbai,python,java,deep learning,c","[mumbai, python, java, deep, learning, c]"
4,c811,trivandrum,"python,java,c,c++","trivandrum,python,java,c,c++","[trivandrum, python, java, c, c++]"
...,...,...,...,...,...
142,c105,chennai,"c++,c,matlab,java,python,assembly","chennai,c++,c,matlab,java,python,assembly","[chennai, c++, c, matlab, java, python, assembly]"
143,c102,mumbai,"python,machine learning,android app developmen...","mumbai,python,machine learning,android app dev...","[mumbai, python, machine, learning, android, a..."
144,c101,hyderabad,"java,javascript,c c++,dbms sql,reactjs","hyderabad,java,javascript,c c++,dbms sql,reactjs","[hyderabad, java, javascript, c, c++, dbms, sq..."
145,c100,chennai,"java programming,r programming,c","chennai,java programming,r programming,c","[chennai, java, programming, r, programming, c]"


In [ ]:
train_grouped=pd.DataFrame(train_df_lowercase.groupby(['location',"skills"])['internship_id'].apply(','.join).reset_index())
#train_grouped.to_csv("train_grouped.csv")
train_grouped

,location,skills,internship_id
0,ahmedabad,"android app developer ,web development",p367
1,ahmedabad,"android,python,sql database,html css,javascrip...","p751,p755"
2,ahmedabad,"aws,alexa,gcp,nodejs,android,java",p334
3,ahmedabad,"c c++,flutter,java,html css,java script,php,da...",p767
4,ahmedabad,"c c++,java,django,node.js,react.js,python,linu...",p729
...,...,...,...
552,trivandrum,"python,c++,machine learning,django rest framew...","p647,p714"
553,trivandrum,"python,c,c++,java,java script",p714
554,trivandrum,"python,django,flask,html,javascript,c,mongodb,...",p375
555,trivandrum,"python,image processing,machine learning,amazo...",p91


In [ ]:
train_grouped=pd.read_csv('train_google_sheet.csv')
train_grouped

,location,skills,internship_id
0,chennai,"c,python,python 3,java,android,android testing...",p10
1,kolkata,"ansible,jenkins,docker,aws,kubernetes,big data...","p10,p94"
2,kolkata,"java,j2ee,python,c,data structures","p10,p94,p318"
3,chennai,"c,c++,matlab,java,html,css,processing js,mysql...","p103,p155"
4,chennai,"python,r programming,machine learning,data ana...",p109
...,...,...,...
552,trivandrum,"c++,r",p95
553,delhi,"java,javascript nodejs express ,c,python 3,c++...","p95,p363"
554,delhi,"trading,java core+swing framework ,c sharp,asp...",p96
555,hyderabad,"c++,c,java,javascript",p97


In [ ]:
train_grouped.dropna(subset=['skills'],inplace=True,)
train_grouped.isna().sum()

location         0
skills           0
internship_id    0
dtype: int64

In [ ]:
train_grouped = train_grouped.reset_index(drop=True)
train_grouped

,location,skills,internship_id
0,chennai,"c,python,python 3,java,android,android testing...",p10
1,kolkata,"ansible,jenkins,docker,aws,kubernetes,big data...","p10,p94"
2,kolkata,"java,j2ee,python,c,data structures","p10,p94,p318"
3,chennai,"c,c++,matlab,java,html,css,processing js,mysql...","p103,p155"
4,chennai,"python,r programming,machine learning,data ana...",p109
...,...,...,...
552,trivandrum,"c++,r",p95
553,delhi,"java,javascript nodejs express ,c,python 3,c++...","p95,p363"
554,delhi,"trading,java core+swing framework ,c sharp,asp...",p96
555,hyderabad,"c++,c,java,javascript",p97


In [ ]:
train_grouped['features'] = train_grouped[['location', 'skills']].apply(lambda x: ','.join(x), axis=1)
train_grouped.head()

,location,skills,internship_id,features
0,chennai,"c,python,python 3,java,android,android testing...",p10,"chennai,c,python,python 3,java,android,android..."
1,kolkata,"ansible,jenkins,docker,aws,kubernetes,big data...","p10,p94","kolkata,ansible,jenkins,docker,aws,kubernetes,..."
2,kolkata,"java,j2ee,python,c,data structures","p10,p94,p318","kolkata,java,j2ee,python,c,data structures"
3,chennai,"c,c++,matlab,java,html,css,processing js,mysql...","p103,p155","chennai,c,c++,matlab,java,html,css,processing ..."
4,chennai,"python,r programming,machine learning,data ana...",p109,"chennai,python,r programming,machine learning,..."


In [ ]:
train_grouped['features_space'] = train_grouped['features'].apply(lambda x: re.split(',| ',x))
train_grouped

,location,skills,internship_id,features,features_space
0,chennai,"c,python,python 3,java,android,android testing...",p10,"chennai,c,python,python 3,java,android,android...","[chennai, c, python, python, 3, java, android,..."
1,kolkata,"ansible,jenkins,docker,aws,kubernetes,big data...","p10,p94","kolkata,ansible,jenkins,docker,aws,kubernetes,...","[kolkata, ansible, jenkins, docker, aws, kuber..."
2,kolkata,"java,j2ee,python,c,data structures","p10,p94,p318","kolkata,java,j2ee,python,c,data structures","[kolkata, java, j2ee, python, c, data, structu..."
3,chennai,"c,c++,matlab,java,html,css,processing js,mysql...","p103,p155","chennai,c,c++,matlab,java,html,css,processing ...","[chennai, c, c++, matlab, java, html, css, pro..."
4,chennai,"python,r programming,machine learning,data ana...",p109,"chennai,python,r programming,machine learning,...","[chennai, python, r, programming, machine, lea..."
...,...,...,...,...,...
552,trivandrum,"c++,r",p95,"trivandrum,c++,r","[trivandrum, c++, r]"
553,delhi,"java,javascript nodejs express ,c,python 3,c++...","p95,p363","delhi,java,javascript nodejs express ,c,python...","[delhi, java, javascript, nodejs, express, , c..."
554,delhi,"trading,java core+swing framework ,c sharp,asp...",p96,"delhi,trading,java core+swing framework ,c sha...","[delhi, trading, java, core+swing, framework, ..."
555,hyderabad,"c++,c,java,javascript",p97,"hyderabad,c++,c,java,javascript","[hyderabad, c++, c, java, javascript]"


In [ ]:
train_grouped['features_space'][0]

['chennai',
 'c',
 'python',
 'python',
 '3',
 'java',
 'android',
 'android',
 'testing',
 'c++']

In [ ]:
vocab=[]
for i in range(len(train_grouped['features'])):
  for j in train_grouped['features_space'][i]:
    if j not in vocab:
      vocab.append(j)
print(len(vocab))
vocab.sort()
vocab[:20]

463


['',
 '.net',
 '3',
 '3.0',
 '3d',
 '4',
 '5',
 '6',
 '8051',
 '8086',
 'adobe',
 'advanced',
 'agile',
 'ahmedabad',
 'ai',
 'ajax',
 'alexa',
 'algo',
 'algorithm',
 'amazon']

In [ ]:
vocab_test=[]
for i in range(len(test_df_lowercase['features'])):
  for j in test_df_lowercase['features_space'][i]:
    if j not in vocab_test:
      vocab_test.append(j)
print(len(vocab_test))
vocab_test.sort()
vocab_test[:20]

284


['',
 '1',
 '10g',
 '3.x',
 '360',
 '3d',
 '3ds',
 '7',
 'adobe',
 'after',
 'aftereffects',
 'ahmedabad',
 'algorithm',
 'alibaba',
 'analysis',
 'analytics',
 'and',
 'android',
 'angular',
 'angularjs']

In [ ]:
l_not_in_test=[]
for i in vocab:
  if i not in vocab_test:
    l_not_in_test.append(i)
l_not_in_test.sort()
print(len(vocab),len(vocab_test),len(l_not_in_test))
l_not_in_test[:10]

463 284 268


['.net', '3', '3.0', '4', '5', '6', '8051', '8086', 'advanced', 'agile']

In [ ]:
l_not_in_train=[]
for i in vocab_test:
  if i not in vocab:
    l_not_in_train.append(i)
l_not_in_train.sort()
print(len(vocab),len(vocab_test),len(l_not_in_train))
l_not_in_train[:10]

463 284 89


['1',
 '10g',
 '3.x',
 '360',
 '3ds',
 '7',
 'after',
 'aftereffects',
 'alibaba',
 'angularjs']

In [ ]:
vocab_train_test=vocab+l_not_in_train


In [ ]:
len(vocab_train_test)

552

In [ ]:
# vectors = []
# for doc in docs:
#     vectors.append([1 if word in doc.lower().split() else 0 for word in words])
# print("vocabulary: ", words)   
# print("vectors: ", vectors)

In [ ]:
train_grouped['features_vector']=train_grouped['features_space'].apply(lambda x: list(1 if word in x else 0 for word in vocab_train_test))

In [ ]:
train_grouped.head(1)

,location,skills,internship_id,features,features_space,features_vector
0,chennai,"c,python,python 3,java,android,android testing...",p10,"chennai,c,python,python 3,java,android,android...","[chennai, c, python, python, 3, java, android,...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,kolkata,"ansible,jenkins,docker,aws,kubernetes,big data...","p10,p94","kolkata,ansible,jenkins,docker,aws,kubernetes,...","[kolkata, ansible, jenkins, docker, aws, kuber...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,kolkata,"java,j2ee,python,c,data structures","p10,p94,p318","kolkata,java,j2ee,python,c,data structures","[kolkata, java, j2ee, python, c, data, structu...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,chennai,"c,c++,matlab,java,html,css,processing js,mysql...","p103,p155","chennai,c,c++,matlab,java,html,css,processing ...","[chennai, c, c++, matlab, java, html, css, pro...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,chennai,"python,r programming,machine learning,data ana...",p109,"chennai,python,r programming,machine learning,...","[chennai, python, r, programming, machine, lea...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
len(train_grouped['features_space'][0])

10

In [ ]:
c=0
for i in train_grouped['features_vector'][0]:
  if i==1:
    c+=1
c

8

In [ ]:
train_grouped.features_vector.shape

(557,)

In [ ]:
test_df_lowercase['features_vector']=test_df_lowercase['features_space'].apply(lambda x: list(1 if word in x else 0 for word in vocab_train_test))
test_df_lowercase

,candidate_id,location,skills,features,features_space,features_vector
0,c93,hyderabad,"python,c,java,android studio","hyderabad,python,c,java,android studio","[hyderabad, python, c, java, android, studio]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,c84,pune,"c,java,android app development,python,c++","pune,c,java,android app development,python,c++","[pune, c, java, android, app, development, pyt...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,c816,hyderabad,"c,java,python,sql,machine learning","hyderabad,c,java,python,sql,machine learning","[hyderabad, c, java, python, sql, machine, lea...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,c813,mumbai,"python,java,deep learning,c","mumbai,python,java,deep learning,c","[mumbai, python, java, deep, learning, c]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,c811,trivandrum,"python,java,c,c++","trivandrum,python,java,c,c++","[trivandrum, python, java, c, c++]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...
142,c105,chennai,"c++,c,matlab,java,python,assembly","chennai,c++,c,matlab,java,python,assembly","[chennai, c++, c, matlab, java, python, assembly]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
143,c102,mumbai,"python,machine learning,android app developmen...","mumbai,python,machine learning,android app dev...","[mumbai, python, machine, learning, android, a...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
144,c101,hyderabad,"java,javascript,c c++,dbms sql,reactjs","hyderabad,java,javascript,c c++,dbms sql,reactjs","[hyderabad, java, javascript, c, c++, dbms, sq...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
145,c100,chennai,"java programming,r programming,c","chennai,java programming,r programming,c","[chennai, java, programming, r, programming, c]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
train_grouped.features_vector=train_grouped.features_vector.apply(lambda x: np.array(x).reshape(1,len(vocab_train_test)))
train_grouped

,location,skills,internship_id,features,features_space,features_vector
0,chennai,"c,python,python 3,java,android,android testing...",p10,"chennai,c,python,python 3,java,android,android...","[chennai, c, python, python, 3, java, android,...","[[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,kolkata,"ansible,jenkins,docker,aws,kubernetes,big data...","p10,p94","kolkata,ansible,jenkins,docker,aws,kubernetes,...","[kolkata, ansible, jenkins, docker, aws, kuber...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,kolkata,"java,j2ee,python,c,data structures","p10,p94,p318","kolkata,java,j2ee,python,c,data structures","[kolkata, java, j2ee, python, c, data, structu...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,chennai,"c,c++,matlab,java,html,css,processing js,mysql...","p103,p155","chennai,c,c++,matlab,java,html,css,processing ...","[chennai, c, c++, matlab, java, html, css, pro...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,chennai,"python,r programming,machine learning,data ana...",p109,"chennai,python,r programming,machine learning,...","[chennai, python, r, programming, machine, lea...","[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
...,...,...,...,...,...,...
552,trivandrum,"c++,r",p95,"trivandrum,c++,r","[trivandrum, c++, r]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
553,delhi,"java,javascript nodejs express ,c,python 3,c++...","p95,p363","delhi,java,javascript nodejs express ,c,python...","[delhi, java, javascript, nodejs, express, , c...","[[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
554,delhi,"trading,java core+swing framework ,c sharp,asp...",p96,"delhi,trading,java core+swing framework ,c sha...","[delhi, trading, java, core+swing, framework, ...","[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
555,hyderabad,"c++,c,java,javascript",p97,"hyderabad,c++,c,java,javascript","[hyderabad, c++, c, java, javascript]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [ ]:
test_df_lowercase.features_vector=test_df_lowercase.features_vector.apply(lambda x: np.array(x).reshape(1,len(vocab_train_test)))
test_df_lowercase

,candidate_id,location,skills,features,features_space,features_vector
0,c93,hyderabad,"python,c,java,android studio","hyderabad,python,c,java,android studio","[hyderabad, python, c, java, android, studio]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,c84,pune,"c,java,android app development,python,c++","pune,c,java,android app development,python,c++","[pune, c, java, android, app, development, pyt...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,c816,hyderabad,"c,java,python,sql,machine learning","hyderabad,c,java,python,sql,machine learning","[hyderabad, c, java, python, sql, machine, lea...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,c813,mumbai,"python,java,deep learning,c","mumbai,python,java,deep learning,c","[mumbai, python, java, deep, learning, c]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,c811,trivandrum,"python,java,c,c++","trivandrum,python,java,c,c++","[trivandrum, python, java, c, c++]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
...,...,...,...,...,...,...
142,c105,chennai,"c++,c,matlab,java,python,assembly","chennai,c++,c,matlab,java,python,assembly","[chennai, c++, c, matlab, java, python, assembly]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
143,c102,mumbai,"python,machine learning,android app developmen...","mumbai,python,machine learning,android app dev...","[mumbai, python, machine, learning, android, a...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
144,c101,hyderabad,"java,javascript,c c++,dbms sql,reactjs","hyderabad,java,javascript,c c++,dbms sql,reactjs","[hyderabad, java, javascript, c, c++, dbms, sq...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
145,c100,chennai,"java programming,r programming,c","chennai,java programming,r programming,c","[chennai, java, programming, r, programming, c]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [ ]:
wordvec_arrays = np.zeros((len(train_grouped.features_vector), len(vocab_train_test))) 
for i in range(len(train_grouped.features_vector)):
    wordvec_arrays[i,:] = train_grouped.features_vector[i].reshape(1,len(vocab_train_test))
train_features_df = pd.DataFrame(wordvec_arrays)
train_features_df.head(2) 

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
train_features_df.shape

(557, 552)

In [ ]:
wordvec_arrays = np.zeros((len(test_df_lowercase.features_vector), len(vocab_train_test))) 
for i in range(len(test_df_lowercase.features_vector)):
    wordvec_arrays[i,:] = test_df_lowercase.features_vector[i].reshape(1,len(vocab_train_test))
test_features_df = pd.DataFrame(wordvec_arrays)
test_features_df.head(2) 

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
test_features_df.shape


(147, 552)

In [ ]:
test_df_lowercase['features_space'][0]

['hyderabad', 'python', 'c', 'java', 'android', 'studio']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(train_features_df, train_grouped.internship_id,
                                             ,random_state=42,test_size=0.2,  
                                             )

In [ ]:
x_train[:2]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551
197,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
286,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
88,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
425,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
196,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
159,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
48,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
y_train.shape

(445,)

#Random forest

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 1000, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [2, 4, 5, 6]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

In [ ]:
rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42)
rf_random.fit(train_features_df, train_grouped.internship_id)
rf_random.best_params_

In [ ]:
# {'bootstrap': False,
#  'max_depth': 10,
#  'max_features': 'auto',
#  'min_samples_leaf': 4,
#  'min_samples_split': 5,
#  'n_estimators': 1800}

In [ ]:
rf_random.score(train_features_df, train_grouped.internship_id)

0.0

In [ ]:
rf2 = RandomForestClassifier(bootstrap=False,
 max_depth= 10,
 max_features= 'auto',
 min_samples_leaf= 4,
 min_samples_split= 5,
 n_estimators= 1800
 )
rf2.fit(train_features_df, train_grouped.internship_id)
rf2.score(train_features_df, train_grouped.internship_id)


0.8043087971274686

In [ ]:
prediction=rf2.predict(test_features_df)
prediction

In [ ]:
#prediction

array(['p592', 'p203', 'p617,p638,p713', 'p314', 'p688', 'p367', 'p580',
       'p474', 'p521', 'p212,p299', 'p149', 'p855', 'p314', 'p521',
       'p855', 'p314', 'p379', 'p544,p607,p699', 'p580', 'p751', 'p203',
       'p765,p839', 'p420,p477', 'p855', 'p855', 'p10,p94,p318',
       'p47,p64,p176', 'p855', 'p592', 'p314', 'p744', 'p86', 'p420,p477',
       'p314', 'p617,p638,p713', 'p474', 'p855', 'p313', 'p3', 'p314',
       'p240', 'p855', 'p186', 'p521', 'p520', 'p240', 'p204', 'p307',
       'p855', 'p520', 'p855', 'p544,p607,p699', 'p10,p94', 'p751',
       'p521', 'p203', 'p318', 'p421', 'p855', 'p256', 'p639,p761',
       'p521', 'p521', 'p833', 'p182', 'p347,p348,p459', 'p580', 'p855',
       'p3', 'p167,p227', 'p544,p607,p699', 'p421', 'p521',
       'p617,p638,p713', 'p67', 'p167,p227', 'p643', 'p855', 'p855',
       'p521', 'p855', 'p855', 'p346', 'p551', 'p474', 'p149', 'p109',
       'p521', 'p650', 'p855', 'p520', 'p364,p392,p450,p476', 'p129',
       'p379', 'p420,p477

In [ ]:
submission_prject_col=pd.DataFrame(columns={'project_ids'},data=prediction)
submission_df=pd.DataFrame(test_df_lowercase['candidate_id'])
submission_df['project_ids']=submission_prject_col['project_ids']

In [ ]:
submission_df[:100]

,candidate_id,project_ids
0,c93,p592
1,c84,p203
2,c816,"p617,p638,p713"
3,c813,p314
4,c811,p688
...,...,...
95,c314,p334
96,c3,p186
97,c299,"p544,p607,p699"
98,c291,p521


In [ ]:
submission_df.to_csv("submission.csv")